In [1]:
import tensorflow as tf
import pandas as pd
import gluonnlp as nlp

In [2]:
import torch
from torch import nn, Tensor
from torch.optim import Optimizer
from torch.utils.data import DataLoader, RandomSampler, DistributedSampler, random_split
from torch.nn import CrossEntropyLoss

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
import torch
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [4]:
bertmodel, vocab = get_pytorch_kobert_model() #get model

using cached model
using cached model


In [5]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [6]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070


In [7]:
dataset_train = nlp.data.TSVDataset("nsmc/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("nsmc/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

gluonnlp.data.TSVDataset 로 로드
gluonnlp.data.BERTSentenceTransform 으로 transform => 토크나이징이 진행된다

In [8]:
#토크나이저
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [9]:
transform = nlp.data.BERTSentenceTransform(tokenizer=tok,
                                           max_seq_length=140,
                                           pad=True,
                                           pair=False)

In [10]:
from_string = transform('별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네')
print(from_string)

(array([   2, 2348,    3,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1], dtype=int32), array(3, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
from torch.utils.data import Dataset, DataLoader

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [12]:
## Setting parameters
max_len = 128 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
import numpy as np
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
#파이토치 텐서로 변환해준다.

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [18]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [19]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)


In [20]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [22]:
from tqdm import tqdm, tqdm_notebook

# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/home/aiffel-dj57/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/4688 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.629084825515747 train acc 0.1875
epoch 1 batch id 201 loss 0.9285391569137573 train acc 0.4510261194029851
epoch 1 batch id 401 loss 0.5567103028297424 train acc 0.5678771820448878
epoch 1 batch id 601 loss 0.40651535987854004 train acc 0.646734608985025
epoch 1 batch id 801 loss 0.4441331624984741 train acc 0.6923767166042447
epoch 1 batch id 1001 loss 0.38147395849227905 train acc 0.7232455044955045
epoch 1 batch id 1201 loss 0.6003241539001465 train acc 0.7437552039966694
epoch 1 batch id 1401 loss 0.2666613459587097 train acc 0.7597028907922913
epoch 1 batch id 1601 loss 0.5532408952713013 train acc 0.7708853841349157
epoch 1 batch id 1801 loss 0.5103179216384888 train acc 0.7796189616879511
epoch 1 batch id 2001 loss 0.2986183762550354 train acc 0.7872626186906547
epoch 1 batch id 2201 loss 0.5946784019470215 train acc 0.7937443207632894
epoch 1 batch id 2401 loss 0.26377755403518677 train acc 0.799003019575177
epoch 1 batch id 2601 loss 0.346682965755462

  0%|          | 0/4688 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.30649450421333313 train acc 0.8125
epoch 2 batch id 201 loss 0.26623180508613586 train acc 0.878731343283582
epoch 2 batch id 401 loss 0.28749722242355347 train acc 0.8794420199501247
epoch 2 batch id 601 loss 0.21214497089385986 train acc 0.8816035773710482
epoch 2 batch id 801 loss 0.4154081642627716 train acc 0.88229556803995
epoch 2 batch id 1001 loss 0.34163832664489746 train acc 0.8852709790209791
epoch 2 batch id 1201 loss 0.45722338557243347 train acc 0.8873334721065779
epoch 2 batch id 1401 loss 0.348471462726593 train acc 0.8892978229835832
epoch 2 batch id 1601 loss 0.432079553604126 train acc 0.8902638975640225
epoch 2 batch id 1801 loss 0.3545224070549011 train acc 0.8912583287062743
epoch 2 batch id 2001 loss 0.2411753386259079 train acc 0.8916479260369815
epoch 2 batch id 2201 loss 0.35148435831069946 train acc 0.8927476147205815
epoch 2 batch id 2401 loss 0.23675966262817383 train acc 0.8938072678050812
epoch 2 batch id 2601 loss 0.132191359996

...

epoch 3 batch id 1 loss 0.45624464750289917 train acc 0.78125
epoch 3 batch id 201 loss 0.14674066007137299 train acc 0.9225746268656716
epoch 3 batch id 401 loss 0.22530721127986908 train acc 0.9212125935162094
epoch 3 batch id 601 loss 0.12346389889717102 train acc 0.9222129783693843
epoch 3 batch id 801 loss 0.25826695561408997 train acc 0.9229088639200999
epoch 3 batch id 1001 loss 0.28246068954467773 train acc 0.9240134865134865
epoch 3 batch id 1201 loss 0.4905385971069336 train acc 0.9260772273105745
epoch 3 batch id 1401 loss 0.357593297958374 train acc 0.9274848322626695
epoch 3 batch id 1601 loss 0.30392494797706604 train acc 0.928540755777639
epoch 3 batch id 1801 loss 0.14707671105861664 train acc 0.929865352581899
epoch 3 batch id 2001 loss 0.0699525773525238 train acc 0.9305347326336831
epoch 3 batch id 2201 loss 0.3827323913574219 train acc 0.931792367105861
epoch 3 batch id 2401 loss 0.18374399840831757 train acc 0.9327233444398167
epoch 3 batch id 2601 loss 0.157196462

  0%|          | 0/4688 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.4996744990348816 train acc 0.84375
epoch 4 batch id 201 loss 0.08056577295064926 train acc 0.9499378109452736
epoch 4 batch id 401 loss 0.09467220306396484 train acc 0.952540523690773
epoch 4 batch id 601 loss 0.17174412310123444 train acc 0.9550228785357737
epoch 4 batch id 801 loss 0.033657558262348175 train acc 0.9561875780274657
epoch 4 batch id 1001 loss 0.0381745770573616 train acc 0.9570117382617382
epoch 4 batch id 1201 loss 0.470498263835907 train acc 0.9577955870108243
epoch 4 batch id 1401 loss 0.19677937030792236 train acc 0.9585340827980015
epoch 4 batch id 1601 loss 0.16007354855537415 train acc 0.9589709556527171
epoch 4 batch id 1801 loss 0.1742366999387741 train acc 0.9596578289838978
epoch 4 batch id 2001 loss 0.012142017483711243 train acc 0.960410419790105
epoch 4 batch id 2201 loss 0.3076639771461487 train acc 0.9610404361653794
epoch 4 batch id 2401 loss 0.1491834670305252 train acc 0.9614743856726364
epoch 4 batch id 2601 loss 0.00722718

  0%|          | 0/4688 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.7392314672470093 train acc 0.84375
epoch 5 batch id 201 loss 0.01995018869638443 train acc 0.9731032338308457
epoch 5 batch id 401 loss 0.07334630936384201 train acc 0.975218204488778
epoch 5 batch id 601 loss 0.11765656620264053 train acc 0.9759255407653911
epoch 5 batch id 801 loss 0.010485641658306122 train acc 0.975187265917603
epoch 5 batch id 1001 loss 0.11940737813711166 train acc 0.9750561938061938
epoch 5 batch id 1201 loss 0.3990212082862854 train acc 0.975541215653622
epoch 5 batch id 1401 loss 0.17361751198768616 train acc 0.9757985367594575
epoch 5 batch id 1601 loss 0.12341782450675964 train acc 0.9759134915677702
epoch 5 batch id 1801 loss 0.014503002166748047 train acc 0.9763672959466962
epoch 5 batch id 2001 loss 0.13301394879817963 train acc 0.9766991504247876
epoch 5 batch id 2201 loss 0.2023618519306183 train acc 0.9771694684234439
epoch 5 batch id 2401 loss 0.12459675222635269 train acc 0.9774703248646397
epoch 5 batch id 2601 loss 0.00333